In [2]:
import pandas as pd
from datetime import datetime
import numpy as np

In [3]:
def dateparse (time_in_secs): 
    return datetime.fromtimestamp(float(time_in_secs))

### Read the data from local file

In [4]:
data = pd.read_csv("2020_01_Gener_BicingNou_ESTACIONS.csv", parse_dates=["last_reported"], date_parser = dateparse)
data = data.drop(axis=1, labels = ["is_installed", "is_renting", "is_returning", "is_charging_station", "status", "ttl", "last_updated"])
data.head()

,station_id,num_bikes_available,num_bikes_available_types.mechanical,num_bikes_available_types.ebike,num_docks_available,last_reported
0,1,21,21,0,23,2019-12-31 23:56:44
1,2,10,9,1,16,2019-12-31 23:57:58
2,3,15,15,0,9,2019-12-31 23:56:20
3,4,8,8,0,13,2019-12-31 23:58:50
4,5,14,11,3,24,2019-12-31 23:58:50


### Before we calculate

We need a list of every station. Although we could use every number from 0 to 500 and then remove stations with 0's, we consider a better idea to create the list from the dataset. The best idea is to retrieve the data from the dataset with the district information as there are no repeated values. Later we will use these same dataset.

In [30]:
info = pd.read_csv("https://raw.githubusercontent.com/jordipuig37/filtered-bicing-data/main/bicing_station_districts.csv")
STATIONS = list(info["station_id"])

In [93]:
YEAR, MONTH = 2020, 1
# useful function to inicialize dictionarys
def make_empty_dic_usage(stations):
    d = {}
    for stat in stations:
        d[stat] = []
    d["datetime"] = []
    return d

def write_to_dic(usage_dict, d, day, h, minute, station, operation, variable):
    if station == -99:
        usage_dict["datetime"].append(datetime(YEAR, MONTH, day, h, minute).isoformat())
    else:
        if operation == "pick":
            toAppend = np.sum(np.abs(d)-d)/2
        elif operation == "park":
            toAppend = np.sum(np.abs(d)+d)/2
        else:
            # pick up and parking
            toAppend = np.sum(np.abs(d))

        usage_dict[station].append(toAppend)

def compute_usage(data, stats, variable, minInterval, operation):
    usage_dict = make_empty_dic_usage(stats)
    stations = stats
    stations.append(-99)
    for station in stations:
        st = data[data.station_id==station]
        for day in range(1, 32):
            aux_days = st[st.last_reported.dt.day == day]
            if minInterval == "days":
                aux_days = aux_days[variable].diff()[1:]
                write_to_dic(usage_dict, aux_days, day, 0, 0, station, operation, variable)
            else:
                for h in range(24):
                    aux_hour = aux_days[aux_days.last_reported.dt.hour == h]
                    if minInterval == "hours":
                        aux_hour = aux_hour[variable].diff()[1:]
                        write_to_dic(usage_dict, aux_hour, day, h, 0, station, operation, variable)
                    else:
                        for half in range(2):
                            aux = aux_hour[aux_hour.last_reported.dt.minute.isin(range(half*30, 30*(half+1)))]
                            aux = aux[variable].diff()[1:]
                            write_to_dic(usage_dict, aux_hour, day, h, 0, station, operation, variable)
    return usage_dict

In [94]:
def w_df(variable, stations):
    picks = pd.DataFrame(compute_usage(data, variable, "days", "pick"))
    pick = picks.melt(id_vars = ["datetime"], var_name = "station", value_vars = stations, value_name = variable)
    
        
    drops = pd.DataFrame(compute_usage(data,stations,  variable, "days", "drop"))
    drops = drops.melt(id_vars = ["datetime"], var_name = "station", value_vars = stations, value_name = variable)
    

In [98]:
variable = "num_bikes_available_types.mechanical"
stations = [1,2]
drops = pd.DataFrame(compute_usage(data, stations, variable, "days", "drop"))
drops = drops.melt(id_vars = ["datetime"], var_name = "station", value_vars = stations[0:-1], value_name = variable)
drops.groupby(["station", drops.datetime.dt.hour])

AttributeError: Can only use .dt accessor with datetimelike values

### And there we go

Now we simply compute the data we want. We can compute the total activity (pickups and dropoffs) or one variable each time. As we are computing the activity of a day, it makes no sense to compare between dropoffs or pickups since each day the bycicles tend to balance (for the case of stations with natural balance or for the action of bycing trucks).

Moreover we will compute the different activity for mechanical and electric bikes.

In [76]:
means_avail = data.groupby(["station_id", data.last_reported.dt.hour]).mean()
means_avail.index.names = ["station", "hour"]
means_avail = means_avail.rename(columns = {"num_bikes_available": "total_avail",
            "num_bikes_available_types.mechanical": "mech_avail",
            "num_bikes_available_types.ebike": "ebike_avail",
            "num_docks_available": "docks_avail"})

In [54]:
mech_usage = compute_usage(data, "num_bikes_available_types.mechanical", "days", "both")
elec_usage = compute_usage(data, "num_bikes_available_types.ebike", "days", "both")

In [55]:
mech_df = pd.DataFrame(mech_usage)
elec_df = pd.DataFrame(elec_usage)

In [74]:
melt_mech = mech_df.melt(id_vars = ["datetime"], var_name = "station", value_vars = STATIONS[0:499], value_name = "mechanical")
melt_elec = elec_df.melt(id_vars = ["datetime"], var_name = "station", value_vars = STATIONS[0:499], value_name = "electrical")
melt_elec = melt_elec.rename(columns={"datetime":"elec-dt", "station":"elec-st"})
melt_mech.head()

,datetime,station,mechanical
0,2020-01-01T00:00:00,1,97.0
1,2020-01-02T00:00:00,1,137.0
2,2020-01-03T00:00:00,1,152.0
3,2020-01-04T00:00:00,1,118.0
4,2020-01-05T00:00:00,1,102.0


In [75]:
# concatenate both dataframes
usage = pd.concat([melt_mech, melt_elec], axis = 1)
usage = usage.drop(["elec-dt", "elec-st"], axis="columns")
usage.head()

,datetime,station,mechanical,electrical
0,2020-01-01T00:00:00,1,97.0,20.0
1,2020-01-02T00:00:00,1,137.0,37.0
2,2020-01-03T00:00:00,1,152.0,43.0
3,2020-01-04T00:00:00,1,118.0,24.0
4,2020-01-05T00:00:00,1,102.0,22.0


In [ ]:
usage.to_csv()

### Enrich the dataset

As we want to plot this data with altair, the easyest way to compute new variables (like one indicating whether the day is weekend) would be now before sending the dataframe to GitHub. One option could be to add the latlon, the district adn/or neighbourhood; thus we would only need to do one `lookup_transform()`.

What we need to do:

* Aggregate the data by districts (sum or mean) in the activity dataframe

* Join both dataframes

In [76]:
info = pd.read_csv("https://raw.githubusercontent.com/jordipuig37/filtered-bicing-data/main/bicing_station_districts.csv")
info.head()

,station_id,name,physical_configuration,lat,lon,altitude,address,post_code,capacity,Latlon,Barri,Districte,Municipi
0,36,"AV. DE LA CATEDRAL, 6",ELECTRICBIKESTATION,41.385062,2.176683,8,"AV. DE LA CATEDRAL, 6",8002,21,"41.38506160000001,2.1766834",el Gòtic,Ciutat Vella,Barcelona
1,53,PL. CARLES PI I SUNYER,ELECTRICBIKESTATION,41.385086,2.174016,10,PL. CARLES PI I SUNYER,8002,21,"41.385086,2.174016",el Gòtic,Ciutat Vella,Barcelona
2,55,"LA RAMBLA, 80",ELECTRICBIKESTATION,41.381428,2.173286,8,"LA RAMBLA, 80",8002,19,"41.3814279,2.1732861",el Gòtic,Ciutat Vella,Barcelona
3,57,"RAMBLA, 2",ELECTRICBIKESTATION,41.376876,2.177225,4,"RAMBLA, 2",8002,25,"41.3768761,2.1772251",el Gòtic,Ciutat Vella,Barcelona
4,126,PG. DE COLOM /VIA LAIETANA,ELECTRICBIKESTATION,41.380628,2.182192,4,PG. DE COLOM /VIA LAIETANA,8002,23,"41.380628,2.1821916",el Gòtic,Ciutat Vella,Barcelona


In [77]:
stationDistrict = dict()
for idx, stat in info.iterrows():
    stationDistrict[stat["station_id"]] = stat["Districte"]

In [78]:
def levels(column):
    levels = set()
    for item in column:
        if item not in levels:
            levels.add(item)
    return levels

districts = list(levels(info["Districte"]))
districts

['Horta-Guinardó',
 'Sant Martí',
 'Gràcia',
 'Sants-Montjuïc',
 'Nou Barris',
 'Ciutat Vella',
 'Sant Andreu',
 'Eixample',
 'Sarrià-Sant Gervasi',
 'Les Corts']

In [82]:
def sum_districts(act, stationDistrict):
    data = dict()
    for station in stationDistrict:
        mech = np.sum(act[act["station"]==station]["mechanical"]) # sum
        elec = np.sum(act[act["station"]==station]["electrical"]) # sum
        if stationDistrict[station] not in data:
            data[stationDistrict[station]] = [0,0]
        data[stationDistrict[station]][0] += mech
        data[stationDistrict[station]][1] += elec

    return data
    

In [83]:
sum_dist = sum_districts(usage, stationDistrict)
sum_dist

{'Ciutat Vella': [210265.0, 39490.0],
 'Eixample': [371421.0, 113291.0],
 'Gràcia': [37984.0, 27321.0],
 'Horta-Guinardó': [21822.0, 16479.0],
 'Les Corts': [36832.0, 24700.0],
 'Nou Barris': [15481.0, 8716.0],
 'Sant Andreu': [57654.0, 22493.0],
 'Sant Martí': [260609.0, 62597.0],
 'Sants-Montjuïc': [106621.0, 36068.0],
 'Sarrià-Sant Gervasi': [41435.0, 32228.0]}

In [86]:
def to_df(dictionary):
    dataframe = {"District":[], "mechanic":[], "electric":[]}
    for d, v in dictionary.items():
        dataframe["District"].append(d)
        dataframe["mechanic"].append(v[0])
        dataframe["electric"].append(v[1])
    return pd.DataFrame(dataframe)

In [87]:
activity = to_df(sum_dist)
activity

,District,mechanic,electric
0,Ciutat Vella,210265.0,39490.0
1,Eixample,371421.0,113291.0
2,Gràcia,37984.0,27321.0
3,Horta-Guinardó,21822.0,16479.0
4,Les Corts,36832.0,24700.0
5,Nou Barris,15481.0,8716.0
6,Sant Andreu,57654.0,22493.0
7,Sant Martí,260609.0,62597.0
8,Sants-Montjuïc,106621.0,36068.0
9,Sarrià-Sant Gervasi,41435.0,32228.0


In [88]:
activity.to_csv("2020_01_district_activity.csv")

## windowed usage

In [6]:
mech_picks = compute_windowed_usage(data, "num_bikes_available_types.mechanical", 6, 22, "pick")
elec_picks = compute_windowed_usage(data, "num_bikes_available_types.ebike", 6, 22, "pick")

mech_drops = compute_windowed_usage(data, "num_bikes_available_types.mechanical", 6, 22, "park")
elec_drops = compute_windowed_usage(data, "num_bikes_available_types.ebike", 6, 22, "park")

In [7]:
df_mech_picks = pd.DataFrame(mech_picks)
df_elec_picks = pd.DataFrame(elec_picks)

df_mech_drops = pd.DataFrame(mech_drops)
df_elec_drops = pd.DataFrame(elec_drops)

In [9]:
melt_mech_picks = df_mech_picks.melt(id_vars = ["datetime"], var_name = "station", value_vars = STATIONS[0:499], value_name = "value")
melt_mech_picks["biketype"] = "mech"
melt_mech_picks["operation"] = "pick"

melt_elec_picks = df_elec_picks.melt(id_vars = ["datetime"], var_name = "station", value_vars = STATIONS[0:499], value_name = "value")
melt_elec_picks["biketype"] = "elec"
melt_elec_picks["operation"] = "pick"

melt_mech_drops = df_mech_drops.melt(id_vars = ["datetime"], var_name = "station", value_vars = STATIONS[0:499], value_name = "value")
melt_mech_drops["biketype"] = "mech"
melt_mech_drops["operation"] = "drop"

melt_elec_drops = df_elec_drops.melt(id_vars = ["datetime"], var_name = "station", value_vars = STATIONS[0:499], value_name = "value")
melt_elec_drops["biketype"] = "elec"
melt_elec_drops["operation"] = "drop"

In [37]:
melt_mech_picks.head()

,datetime,station,value,biketype,operation
0,2020-01-01T00:00:00,1,39.0,mech,pick
1,2020-01-02T00:00:00,1,54.0,mech,pick
2,2020-01-03T00:00:00,1,83.0,mech,pick
3,2020-01-04T00:00:00,1,48.0,mech,pick
4,2020-01-05T00:00:00,1,45.0,mech,pick


In [45]:
long_form_windowed = melt_mech_picks.append(melt_elec_picks).append(melt_mech_drops).append(melt_elec_drops)
long.head()

,datetime,station,value,biketype,operation
0,2020-01-01T00:00:00,1,39.0,mech,pick
1,2020-01-02T00:00:00,1,54.0,mech,pick
2,2020-01-03T00:00:00,1,83.0,mech,pick
3,2020-01-04T00:00:00,1,48.0,mech,pick
4,2020-01-05T00:00:00,1,45.0,mech,pick


In [46]:
long_form_windowed.to_csv("2020_01_windowed_activity_long.csv")

### Wide form

In [20]:
melt_mech_picks2 = df_mech_picks.melt(id_vars = ["datetime"], var_name = "station", value_vars = STATIONS[0:499], value_name = "mech_picks")
melt_mech_picks2.set_index(["datetime", "station"], inplace = True)

melt_elec_picks2 = df_elec_picks.melt(id_vars = ["datetime"], var_name = "station", value_vars = STATIONS[0:499], value_name = "elec_picks")
melt_elec_picks2.set_index(["datetime", "station"], inplace = True)

melt_mech_drops2 = df_mech_drops.melt(id_vars = ["datetime"], var_name = "station", value_vars = STATIONS[0:499], value_name = "mech_drops")
melt_mech_drops2.set_index(["datetime", "station"], inplace = True)

melt_elec_drops2 = df_elec_drops.melt(id_vars = ["datetime"], var_name = "station", value_vars = STATIONS[0:499], value_name = "elec_drops")
melt_elec_drops2.set_index(["datetime", "station"], inplace = True)


In [24]:
wide_windowed = melt_mech_drops2.join(melt_elec_drops2).join(melt_elec_picks2).join(melt_mech_picks2)
wide_windowed.head()

,,mech_drops,elec_drops,elec_picks,mech_picks
datetime,station,,,,
2020-01-01T00:00:00,1,31.0,6.0,4.0,39.0
2020-01-02T00:00:00,1,71.0,16.0,16.0,54.0
2020-01-03T00:00:00,1,52.0,19.0,23.0,83.0
2020-01-04T00:00:00,1,39.0,11.0,11.0,48.0
2020-01-05T00:00:00,1,41.0,11.0,10.0,45.0


In [25]:
wide_windowed.to_csv("2020_01_windowed_activity_wide.csv")